** Scratch pad for tests **
Expects HF models to be served with TGI - use `make run_tgi` to launch TGi from docker container. 
Launch api server with `python chat_api.py`

In [ ]:
import requests
from pydantic import BaseModel
from typing import Optional
import json

In [ ]:
url = 'http://localhost:8884/create_agent_session'
response = requests.get(url)
response


In [ ]:
session=response.content.decode('utf-8')
session = json.loads(session)
session_id=session['session_id']
data = {"user_input":"What are the top 3 tourist destiantions in Sydney?","session_id":session_id}
#data
print(data)


In [ ]:
%%timeit -n 1 -r 1
url_chat = 'http://localhost:8884/chat_with_agent'
with requests.post(url_chat, json=data, stream=True) as response:
    for x in response.iter_content(chunk_size=None):
        print (x.decode('utf-8'), end="", flush=True)


In [ ]:
%%timeit -n 5 -r 1
url_chat = 'http://localhost:8884/chat_with_agent'
with requests.post(url_chat, json=data) as response:
    print (response.content)

In [ ]:
import aiohttp
import asyncio

In [ ]:
url_chat = 'http://localhost:8884/chat_with_agent'
async with aiohttp.ClientSession() as session:
    async with session.post(url_chat, json=data) as response:
        async for x in response.content.iter_any():
            print (x.decode('utf-8'), end="", flush=True)

In [ ]:
data['user_input'] = "You are good at finding and correcting spelling mistakes. Please correct the spelling mistakes to make it proper English: 'Please Sow mW capiorel for\n\nhelio.\nof gourrel Sundiicpes and hore\nmeade. deseer le'"

In [ ]:
url_chat = 'http://localhost:8884/chat_with_agent'
async with aiohttp.ClientSession() as session:
    async with session.post(url_chat, json=data) as response:
        async for x in response.content.iter_any():
            print (x.decode('utf-8'), end="", flush=True)

In [ ]:
from omegaconf import OmegaConf
from griptape.rules import Rule, Ruleset
from typing import List, Dict
from dataclasses import dataclass
from attr import field, define, Factory

@dataclass
class my_configs:
    rule_strings: List[str] = field(default=Factory(list), kw_only=True)

    @property
    def rules(self):
        rule_list=[]
        for rule in self.rule_strings:
            rule_list.append[Rule(rule)]
        return rule_list
    @rules.setter
    def rules(self,rules:List[str]):
        self.rule_strings = self.rules
        


In [ ]:
test =my_configs(rule_strings=['You are a total tool','You are not a tool'])

_config = OmegaConf.create(test)
OmegaConf.save(_config,"conf.yaml")

In [ ]:
_config.keys()

In [ ]:
a=OmegaConf.load("conf.yaml")
parsed_a = OmegaConf.structured(a,my_configs)

In [ ]:
from omegaconf import OmegaConf
from data.chat_data_classes import ModelDriverConfiguration
a=ModelDriverConfiguration(model='http://localhost:8080',pretrained_tokenizer="/home/mtman/Documents/Repos/yakwith.ai/models/Mistral-7B-OpenOrca")
conf = OmegaConf.create(a)
OmegaConf.save(conf,'default_model_driver.yaml')
conf

In [ ]:
x = OmegaConf.load('default_model_driver.yaml')

In [ ]:
x.keys()

In [ ]:
type(x), x.token

In [ ]:
for thing in x:
    print(getattr(x,thing))

In [ ]:
new_conf = ModelDriverConfiguration()

In [ ]:
type(new_conf)

In [ ]:
new_conf.from_omega_conf(x)

In [ ]:
new_conf

In [ ]:
from voice_chat.agents.yak_agent import YakAgent

In [2]:
from data.chat_data_classes import RuleList
from griptape.rules import Rule
from omegaconf import OmegaConf

In [4]:
conf=OmegaConf.load('configs/rules/agent/default_rule_set.yaml')
cls=RuleList().from_omega_conf(conf)

AttributeError: 'RuleList' object attribute 'name' is read-only

In [3]:
for key in conf:
    if hasattr(cls, key):
        _value = getattr(conf, key)
        if key == "rules":
            ret = []
            for rule_string in _value:
                ret.append(Rule(rule_string))
            setattr(cls, key, ret)
        else:
            setattr(cls, key, _value)

In [8]:
cls.name

AttributeError: 'NoneType' object has no attribute 'name'

In [5]:
conf

{'name': 'CommonRules', 'rules': ['You can not substitute any ingredients', 'If a meal contains meat, you can replace meat with Tofu or Mushrooms', "If you do not know the answer to a customers questions, say 'I have to ask the manager, please give me a second.'", 'All meals and drinks can be ordered for takeaway.']}

In [30]:
a

Rule(value='Hello')

In [1]:
from omegaconf import OmegaConf
from data.chat_data_classes import Menu, MenuList
import attr

In [2]:
breakfast_menu = Menu('Twist','rp 7 I Ss | y\nIW\nESPRESSO & WINE\n\nWINTER BRUNCH MENU AVAILABLE UNTIL 1.30PM\n\n \n\n \n\nRUSTIC TOAST - 8.5\nSourdough or Ciabatta.\nFig & lemon jam, orange marmalade, strawberry\njam, peanut butter or vegemite.\n\nHOT CHEESE - 9.5\n\nWith tomato & oregano on sourdough.\n\nFRUIT TOAST - 8.5\nApricots, dates, fig & nuts, toasted with butter.\n\nGRANOLA CUP GF - 10.5\nHouse-made granola, greek yoghurt, strawberry\njam and strawberries.\n\nACAI BOWL GF - 17.5\nHouse-made granola, banana, strawberry.\nadd peanut butter 2\n\nITALIAN OMELETTE - 23\nTomato, mozarella, topped with basil & walnut\npesto, mixed leaves, ciabatta.\n\nMEXICAN EGGS - 23\nMixed beans cooked with chilli, garlic, onion and\nherbs in rich tomato sauce, baked with\nmozzerella, two poached eggs, ciabatta.\n\nEGGS BENEDICT - 26\nTwo poached eggs, spinach, and hollandaise sauce\nserved on sourdough.\nChoice of : salmon, bacon or ham.\n\nEGGS POACHED OR FRIED - 13\n\nServed on sourdough or ciabatta.\n\nEXTRAS:\n\nSalmon, Gourmet Pork Sausages 6\nBacon, Haloumi 5\nAvocado, Grilled Tomato 4\nFree-Range Egg 3\nGluten Free Bread 2\n\n \n\nTWIST BIG BREAKFAST ~ 28\nTwo poached or fried eggs, gourmet pork\nsausages, haloumi, grilled tomato, spinach,\nsourdough,\n\nBREKKY ROLL ~ 13\nBacon & egy on a milk bun.\nSauce: tomato, BBQ, chipotle mayo or aioli.\nadd cheese 1 smashed avo 2\n\nTWIST WINTER BUN ~ 17\nBacon or haloumi, egg, tomato, caramelised\nonion, mixed leaves, melted cheese, aioli.\nadd smashed avo 2\n\nAVO BRUSCHETTA ~ 22\nOn sourdough, heirloom tomatoes, fetta,\nsecret salt, basil & walnut pesto.\nadd poached egg 3\n\nSOUP OF THE DAY - 16\nWith rustic bread.\n\nPlease Sow mW capiorel for\n\nhelio.\nof gourrel Sundiicpes and hore\nmeade. deseer le\n\nWWW.TWISTESPRESSO.COM\n\nPlease order at the counter = 10% surcharge Sundays & 15% Public I Tolidays\n\n \n\x0c')

In [3]:
breakfast_menu

Menu(cafe_id='Twist', menu='rp 7 I Ss | y\nIW\nESPRESSO & WINE\n\nWINTER BRUNCH MENU AVAILABLE UNTIL 1.30PM\n\n \n\n \n\nRUSTIC TOAST - 8.5\nSourdough or Ciabatta.\nFig & lemon jam, orange marmalade, strawberry\njam, peanut butter or vegemite.\n\nHOT CHEESE - 9.5\n\nWith tomato & oregano on sourdough.\n\nFRUIT TOAST - 8.5\nApricots, dates, fig & nuts, toasted with butter.\n\nGRANOLA CUP GF - 10.5\nHouse-made granola, greek yoghurt, strawberry\njam and strawberries.\n\nACAI BOWL GF - 17.5\nHouse-made granola, banana, strawberry.\nadd peanut butter 2\n\nITALIAN OMELETTE - 23\nTomato, mozarella, topped with basil & walnut\npesto, mixed leaves, ciabatta.\n\nMEXICAN EGGS - 23\nMixed beans cooked with chilli, garlic, onion and\nherbs in rich tomato sauce, baked with\nmozzerella, two poached eggs, ciabatta.\n\nEGGS BENEDICT - 26\nTwo poached eggs, spinach, and hollandaise sauce\nserved on sourdough.\nChoice of : salmon, bacon or ham.\n\nEGGS POACHED OR FRIED - 13\n\nServed on sourdough or cia

In [4]:
OmegaConf.save(breakfast_menu,'twist_breakfast_menu.yaml')

In [5]:
conf = Menu.from_omega_conf(OmegaConf.load('menus/twist_breakfast_menu.yaml'))

In [9]:
menu_list = MenuList('twist',[breakfast_menu,breakfast_menu])

In [11]:
OmegaConf.save(menu_list,'twist_all_menus.yaml')

In [2]:
list_menus = MenuList.from_omega_conf(OmegaConf.load('twist_all_menus.yaml'))

breakfast
breakfast


ValueError: Menu names must be unique. "breakfast" was repeated for twist

In [3]:
list_menus

MenuList(cafe_id='twist', menus=[Menu(cafe_id='Twist', menu='rp 7 I Ss | y\nIW\nESPRESSO & WINE\n\nWINTER BRUNCH MENU AVAILABLE UNTIL 1.30PM\n\n \n\n \n\nRUSTIC TOAST - 8.5\nSourdough or Ciabatta.\nFig & lemon jam, orange marmalade, strawberry\njam, peanut butter or vegemite.\n\nHOT CHEESE - 9.5\n\nWith tomato & oregano on sourdough.\n\nFRUIT TOAST - 8.5\nApricots, dates, fig & nuts, toasted with butter.\n\nGRANOLA CUP GF - 10.5\nHouse-made granola, greek yoghurt, strawberry\njam and strawberries.\n\nACAI BOWL GF - 17.5\nHouse-made granola, banana, strawberry.\nadd peanut butter 2\n\nITALIAN OMELETTE - 23\nTomato, mozarella, topped with basil & walnut\npesto, mixed leaves, ciabatta.\n\nMEXICAN EGGS - 23\nMixed beans cooked with chilli, garlic, onion and\nherbs in rich tomato sauce, baked with\nmozzerella, two poached eggs, ciabatta.\n\nEGGS BENEDICT - 26\nTwo poached eggs, spinach, and hollandaise sauce\nserved on sourdough.\nChoice of : salmon, bacon or ham.\n\nEGGS POACHED OR FRIED -